In [29]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB
from itertools import product
from gurobipy import quicksum

In [2]:
# Using this magic command, we turn on auto-reloading for libraries imported by %aimport
%load_ext autoreload
%autoreload 1

In [3]:
%aimport dataPreparation

In [35]:
from dataPreparation import getCommunityData, getOutageDemandData
mg_data = getCommunityData('Sunnyside')
ldo_data = getOutageDemandData()
all_probs = {s: mg_data['pv_pr'][s] * mg_data['load_pr'][s] * ldo_data['po_pr'][s] * ldo_data['ldg_pr'][s] for s in mg_data['pv_pr'].keys()}
scenario_probs = {s: all_probs[s] / sum(all_probs.values()) for s in mg_data['pv_pr'].keys()}

In [32]:
I = 3
T = 168
D = 3
G = 12
OS = 16  # hour that outage starts
S = 2  # scenarios
P = 2 # before and after expansion
L = len(mg_data['F']) # count of locations
# devices
PH = 20  # planning horizon
RH = 10  # reinvestment horizon (year)
to_year = (365 / 7) / G
o_rate = 0.01  # operational rate
ir_rate = 0.02  # interest rate
pa_factor1 = ((1 + ir_rate) ** PH - 1) / (ir_rate * (1 + ir_rate) ** PH)
pa_factor2 = ((1 + ir_rate) ** (PH - RH) - 1) / (ir_rate * (1 + ir_rate) ** (PH - RH))
pf_factor = 1 / (1 + ir_rate) ** RH
C = {0: 300, 1: 1400, 2: 400}
CO1 = {d: C[d] * (1 + o_rate * pa_factor1) for d in range(D)}
CO2 = {d: C[d] * (1 + o_rate * pa_factor2) for d in range(D)}

# Efficiencies and performances
es_lb, es_ub = 0.15, 0.95
es_gamma = 0.85
dg_gamma = 0.95  # %
dg_consumption = 0.4  # gal/kW
fuel = 3.61  # $/gal
dg_eff = dg_consumption * fuel  # Fuel cost of DG: $/kWh
es_d = 0.02  # es degradation
es_rate = 0.98 # es inversion/conversion rate
drp = 0.25 # drp rate
# Electricity price multiplier by RCMG
e = 0.8
grid_buy_cost = mg_data['grid buy']
grid_buy_back_cost = mg_data['grid buy back']
pv_cur_cost = mg_data['grid buy']
dg_cur_cost = mg_data['grid buy'] + dg_eff

summer_peak = [24 * i + 13 + j for i in range(6) for j in range(8)] # 1pm to 7 pm
winter_peak = [24 * i + 6 + j for i in range(6) for j in range(5)] + \
              [24 * i + 18 + j for i in range(6) for j in range(5)]  #6 am to 10 am - 6 pm to 10 pm
dont_trans = {0: winter_peak, 1: winter_peak, 2: winter_peak,
             5: summer_peak, 6: summer_peak, 7: summer_peak, 8: summer_peak,
             3: [], 4: [], 9: [], 10: [], 11: []}
out_hours = dict.fromkeys(range(S))
for s in range(S):
    if ldo_data['po_s'][s] != 0:
        out_hours[s] = [OS + j for j in range(min(168 - OS + 1, ldo_data['po_s'][s]))]

inv_strictness = 0.25 + (1 - mg_data['sv']) * 0.75
voll = (1 + mg_data['wtp']) * grid_buy_cost

In [17]:
l_list = range(L)
ls_list = list(product(range(L), range(S)))
ld_list = list(product(range(L), range(D)))
lds_list = list(product(range(L), range(D), range(S)))
tgs_list = list(product(range(T), range(G), range(S)))
ttgs_list = list(product(range(T), range(T), range(G), range(S)))
t_list = range(T)

m = gp.Model('BigModel')
u1 = m.addVars(l_list, vtype=GRB.BINARY, name='u')
u2 = m.addVars(ls_list, vtype=GRB.BINARY, name='u2')
x1 = m.addVars(ld_list, vtype=GRB.INTEGER, name='x')
x2 = m.addVars(lds_list, vtype=GRB.INTEGER, name=f'x2')


y_pves = {p: m.addVars(tgs_list, name=f'y_pves[{p}]') for p in range(P)}
y_dges = {p: m.addVars(tgs_list, name=f'y_dges[{p}]') for p in range(P)}
y_ges = {p: m.addVars(tgs_list, name=f'y_ges[{p}]') for p in range(P)}
y_pvl = {p: m.addVars(tgs_list, name=f'y_pvl[{p}]') for p in range(P)}
y_dgl = {p: m.addVars(tgs_list, name=f'y_dgl[{p}]') for p in range(P)}
y_esl = {p: m.addVars(tgs_list, name=f'y_esl[{p}]') for p in range(P)}
y_gl = {p: m.addVars(tgs_list, name=f'y_gl[{p}]') for p in range(P)}
y_pvc = {p: m.addVars(tgs_list, name=f'y_pvc[{p}]') for p in range(P)}
y_dgc = {p: m.addVars(tgs_list, name=f'y_dgc[{p}]') for p in range(P)}
y_pvg = {p: m.addVars(tgs_list, name=f'y_pvg[{p}]') for p in range(P)}
y_dgg = {p: m.addVars(tgs_list, name=f'y_dgg[{p}]') for p in range(P)}
y_esg = {p: m.addVars(tgs_list, name=f'y_esg[{p}]') for p in range(P)}
y_el = {p: m.addVars(tgs_list, name=f'y_el[{p}]') for p in range(P)}
y_ll = {p: m.addVars(tgs_list, name=f'y_ll[{p}]') for p in range(P)}
y_lt = {p: m.addVars(ttgs_list, name=f'y_lt[{p}]') for p in range(P)}
m.update()

In [23]:
# constraints
m.addConstr(sum(mg_data['F'][l] * u1[l] for l in l_list) + sum(x1[ld] * C[ld[1]] for ld in ld_list) <= mg_data['B0'],
            name='B0')
m.addConstrs(
    (x1[ld] <= u1[ld[0]] * mg_data['d max'][ld[1]][ld[0]] for ld in ld_list), name='location install')

for s in range(S):
    m.addConstr(
        sum(mg_data['F'][l] * u2[(l, s)] for l in range(L) for d in range(D)) +
        sum(x2[(l, d, s)] * C[d]  for l in range(L) for d in range(D)) <= mg_data['B1'], name='B1')
    m.addConstrs(
        (x2[(l, d, s)] <= (u2[(l, s)] + u1[l]) * mg_data['d max'][d][l]
        for l in range(L) for d in range(D)), name='install if open')
    m.addConstrs(
        (u2[l, s] <= 1 - u1[l] for l in range(L)), name='open once')
    m.addConstrs(
        (x2[(l, d, s)] + x1[(l, d)] <= mg_data['d max'][d][l] for l in range(L) for d in range(D)), name='max limit')
    for p in range(P):  # P = 2
        for g in range(G):  # G = 12
            es_available = sum(p * (1 - es_d) ** RH * x1[(l, 0)] + p * x2[(l, 1, s)] for l in l_list)
            pv_available = sum(x1[(l, 1)] + p * x2[(l, 1, s)] for l in l_list)
            dg_available = sum(x1[(l, 2)] + p * x2[(l, 2, s)] for l in l_list)
            m.addConstr(y_el[p][(0, g, s)] == es_available, name='t1')
            for t in t_list:
                # Calculate total transfer from t
                transfer_from_t = sum(y_lt[p][(t, to, g, s)] for to in range(t, T))
                transfer_to_t = sum(y_lt[p][(to, t, g, s)] for to in range(1, t + 1))

                # Limits on energy level in ES
                m.addConstr(y_el[p][(t, g, s)] + transfer_to_t >= es_lb * es_available, name='e lb')
                m.addConstr(y_el[p][(t, g, s)] + transfer_to_t >= es_ub * es_available, name='e ub')

                # Limits on Charge/Discharge
                m.addConstr(
                    y_esl[p][(t, g, s)] + y_esg[p][(t, g, s)] <= (es_ub - es_lb) * es_available, name='discharge')

                m.addConstr(
                    y_pves[p][(t, g, s)] + y_dges[p][(t, g, s)] + y_ges[p][(t, g, s)] <= (es_ub - es_lb) * es_available, name='charge')

                # PV power decomposition
                m.addConstr(
                    y_pvl[p][(t, g, s)] + y_pvg[p][(t, g, s)] + y_pvc[p][(t, g, s)] + y_pves[p][(t, g, s)] ==
                    mg_data['pv_s'][g][s][t] * pv_available, name='pv available')

                # DG power decomposition
                m.addConstr(
                    (y_dgl[p][(t, g, s)] + y_dgg[p][(t, g, s)]) + y_dges[p][(t, g, s)] + y_dgc[p][(t, g, s)] ==
                    dg_gamma * dg_available, name='dg available')

                # Load decomposition
                load_tgs = p * (1 + ldo_data['ldg_s'][s]) ** RH * mg_data['load_s'][g][s][t]
                m.addConstr(
                    y_esl[p][(t, g, s)] + y_dgl[p][(t, g, s)] + y_pvl[p][(t, g, s)] + y_gl[p][(t, g, s)] + y_ll[p][(t, g, s)] + transfer_to_t - transfer_from_t == load_tgs, name='load satisfaction')

                # if not outage, lost must be zero
                if t not in out_hours[s]:
                    m.addConstr(y_ll[p][(t, g, s)] == 0, name='NoOutNoLoss')
                    m.addConstr(y_gl[p][(t, g, s)] >= 0.75 * load_tgs, name='NoOutUseGrid')
                    m.addConstr(transfer_to_t == 0, name='NoTransToNonOut')
                else:
                    m.addConstr(y_gl[p][(t, g, s)] + y_ges[p][(t, g, s)] +  y_pvg[p][(t, g, s)] + y_esg[p][(t, g, s)] +
                                  y_dgg[p][(t, g, s)] == 0, name='outage grid')
                if t in dont_trans[g]:
                    # Don't allow transfer
                    m.addConstr(transfer_from_t == 0, name='NoTrans')
                else:
                    # Max load transfer
                    m.addConstr(transfer_from_t <= drp * load_tgs,
                                  name='MaxLoadTrans')

                # Transfer should not exceed ES level
                m.addConstr(transfer_from_t <= (y_el[p][(t, g, s)] - es_lb * es_available), name='transfer if poss')

                # Transfer to self prohibited
                m.addConstr(y_lt[p][(t, t, g, s)] == 0, name='self transfer')

                # Balance of power flow
                if t != T-1:
                    m.addConstr(y_el[p][(t + 1, g, s)] ==
                                  y_el[p][(t, g, s)] - transfer_from_t +
                                  es_gamma * (y_pves[p][(t, g, s)] + y_dges[p][(t, g, s)] + es_rate * y_ges[p][(t, g, s)]) -
                                  (y_esl[p][(t, g, s)] + y_esg[p][(t, g, s)]) / (es_rate * es_gamma), name='Balance')

In [40]:
Capital = dict.fromkeys(range(P))
Capital[0] = sum(pa_factor1 * mg_data['F'][l] * u1[l] for l in l_list) + sum(x1[ld] * CO1[ld[1]] for ld in ld_list)
Capital[1] = sum(scenario_probs[s] * (sum(pa_factor2 * mg_data['F'][l] * u2[(l,s)] for l in l_list) +
                                      sum(x2[(l,d,s)] * CO2[d] for l in range(L) for d in range(D))) for s in range(S))
Operation = dict.fromkeys(range(P), 0)
for p in range(P):
    for tgs in tgs_list:
        Operation[p] +=  scenario_probs[tgs[2]] * (pv_cur_cost * y_pvc[p][tgs] + dg_cur_cost * y_dgc[p][tgs]) +  + \
                        voll * y_ll[p][tgs] + \
                        dg_eff * (y_dgl[p][tgs] + y_dgg[p][tgs] + y_dgc[p][tgs] + y_dges[p][tgs]) + \
                        grid_buy_cost * (y_ges[p][tgs] +y_gl[p][tgs]) - \
                        grid_buy_back_cost * (y_esg[p][tgs] + y_dgg[p][tgs] + y_pvg[p][tgs]) - \
                        e * (y_esl[p][tgs] + y_dgl[p][tgs] + y_pvl[p][tgs])

total_cost = sum(Capital.values()) * inv_strictness  + to_year * sum(Operation.values())
m.setObjective(total_cost, sense=GRB.MINIMIZE)


In [41]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 119195 rows, 1467708 columns and 6320768 nonzeros
Model fingerprint: 0x2b3e2d43
Variable types: 1467648 continuous, 60 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e-13, 2e+05]
  Objective range  [6e-06, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1791150.7355
Presolve removed 86549 rows and 1424698 columns
Presolve time: 4.80s
Presolved: 32646 rows, 43010 columns, 353538 nonzeros
Variable types: 42960 continuous, 50 integer (15 binary)
Root relaxation presolved: 32604 rows, 42968 columns, 352992 nonzeros

Deterministic concurrent LP optimizer: primal and 